# Prompt Caching

Gonna load all of the app data in and ask it questions with follow ups

https://ai.google.dev/gemini-api/docs/caching?lang=python



In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
%pip install -qU google-generativeai python-dotenv pandas

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import google.generativeai as genai
from google.generativeai import caching
import datetime
import time
from dotenv import load_dotenv
from utils import get_app_store_data, get_context

load_dotenv()



True

In [12]:
# Get app store data
df = get_app_store_data()

# Get 150,000 tokens of context
app_data_str, app_df = get_context(150000, df)


In [9]:
system_prompt = """You are an App Store Data Analyzer. You should analyze the provided app store data and answer the user's questions.

- Only use the App Store Data provided in your context.
- Do not answer questions you are not confident in answering because the answer can't be found in the provided context."""

In [13]:
print(app_data_str)

App Name:  Japanese Gacha
Size: 73.7431640625 MB
Price: 0.0 USD
Rating Count: 3
User Rating: 4.0
Version: 1.4.1
Genre: Games
Description: ◯ How to play

1. To draw the Gacha put the coin!
2. It comes out memories from the capsule and pull the Gacha!
3. Try to complete the picture book collecting memories!
App Name: Scary Doll
Size: 15.71484375 MB
Price: 3.99 USD
Rating Count: 18
User Rating: 2.0
Version: 1
Genre: Games
Description: Roses are red
Violets are blue
Don't close your eyes
It's coming for you
App Name: Mia and me  Free the Unicorns
Size: 584.67578125 MB
Price: 2.99 USD
Rating Count: 45
User Rating: 4.0
Version: 1.21
Genre: Games
Description: Life seems just perfect in the Elven Kingdom. But when the evil Gargona captures Mia's friends, you have to help her save them! Join Mia and her best friend, the winged unicorn Onchao on this epic adventure through Centopia! Ride and fly over beautiful landscapes - from the Elven Crater to the Dragon Caves - and save all unicorns and Pri

In [16]:
# Create a cache with a 5 minute TTL
cache = caching.CachedContent.create(
    model='models/gemini-1.5-pro-001',
    display_name='App Store Data',  # used to identify the cache
    system_instruction=system_prompt,
    contents=[app_data_str],
    ttl=datetime.timedelta(minutes=5),
)

In [18]:
# Construct a GenerativeModel which uses the created cache.
model = genai.GenerativeModel.from_cached_content(cached_content=cache)

# Query the model
response = model.generate_content([("What are the top 3 most popular apps?")])


print(response.usage_metadata)
print(response.text)

prompt_token_count: 157120
cached_content_token_count: 157109
candidates_token_count: 92
total_token_count: 157212

The provided app data lists the "Rating Count" for each app, which can be interpreted as a measure of popularity. Based on that:

1. **Sudoku** (Rating Count: 359832)
2. **Shazam - Discover music, artists, videos & lyrics** (Rating Count: 402925)
3. **Hangman** (Rating Count: 42316) 



In [19]:
# Get the top 3 apps with the most ratings
top_apps = app_df.sort_values(by="rating_count_tot", ascending=False).head(3)
top_apps

,id,name,size,currency,price,rating_count_tot,user_rating,ver,prime_genre,app_desc
114,500116670,Clear Vision 17,36.125000,USD,0.99,541693,4.5,1.1.3,Games,#1 Sniper game on the App Store worldwide!\n\n...
313,440045374,DragonVale,145.983398,USD,0.00,503230,4.5,3.15.0,Games,Discover the most popular Dragon Collecting ga...
287,284993459,Shazam Discover music artists videos lyrics,140.279297,USD,0.00,402925,4.0,11.0.3,Music,Shazam is one of the world’s most popular apps...


In [18]:
print(app_str)

App Name: 259
Size: 10 MB
Price: 2.99 USD
Rating Count: 138
User Rating: 4.5
Version: 1.2
Genre: Games
Description: "It's smart, it's pretty... what makes it shine is the dark comedy found in killing cartoon animals just to satisfy some arbitrary numerical requirements." - 8/10 by DESTRUCTOID

Divide By Sheep is a math puzzler about the Grim Reaper's devious plan to make some sheepy friends. Grim can only befriend the dead, so he decided to unleash a great flood and drag the sheep under. Now stranded on small islands and sinking fast, the sheep need you to help them reach safety.

It plays similar to classic counting games with dark humor where you need to cross a road or river, and can't have specific combinations of sheep, wolves, etc on the same raft. Here though the results are intentionally filled with gore and blood (unless you turn it off in the settings). It's a funny, dark math puzzle game. 

You save the little animals by solving clever math puzzles. Divide By Sheep is a game